In [1]:
import sys
sys.path.insert(0,'../models/')
sys.path.insert(0,'../datasets/')
sys.path.insert(0,'..')

import pandas as pd
import numpy as np
import json
from subprocess import Popen, PIPE, STDOUT
import re
from collections import defaultdict
from utils.info import get_db_bounds
from datasets import propbankbr_arg2se

import tensorflow as tf
# import tqdm
from models import PropbankEncoder
import config 

INPUT_DIR = '../datasets/binaries/1.0/'
PROPBANK_WAN50_PATH = '{:}wan50/deep_wan50.pickle'.format(INPUT_DIR)
PEARL_SRLEVAL_PATH = '../srlconll04/srl-eval.pl'

/Users/Varela/.venv/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/Varela/.venv/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<h1><center>Structured Prediction Network CWIS SRL (BR)</center></h1>

<center>In this notebook we solve the semantic role labeling task using structured predictions networks.</center>

## 1. Builds a "human friendly" version of the dataset

In [2]:
dfgs = pd.read_csv('../datasets/csvs/1.0/gs.csv', index_col=0, sep=',', encoding='utf-8')
column_files = [
    '../datasets/csvs/1.0/column_chunks/chunks.csv',
    '../datasets/csvs/1.0/column_predmarker/predicate_marker.csv',
    '../datasets/csvs/1.0/column_shifts_ctx_p/form.csv',
    '../datasets/csvs/1.0/column_shifts_ctx_p/gpos.csv',
    '../datasets/csvs/1.0/column_shifts_ctx_p/lemma.csv',
    '../datasets/csvs/1.0/column_iob/iob.csv',
    '../datasets/csvs/1.0/column_t/t.csv'
]

for col_f in column_files:
    _df = pd.read_csv(col_f, index_col=0, encoding='utf-8')
    dfgs = pd.concat((dfgs, _df), axis=1)

DISPLAY_COLUMNS = ['ID', 'P', 'FORM', 'ARG', 'T', 
                   'CHUNK_ID', 'CHUNK_START', 'CHUNK_FINISH', 'CHUNK_LEN', 'CHUNK_CANDIDATE_ID']            
dfgs[DISPLAY_COLUMNS].head(33)    

,ID,P,FORM,ARG,T,CHUNK_ID,CHUNK_START,CHUNK_FINISH,CHUNK_LEN,CHUNK_CANDIDATE_ID
INDEX,,,,,,,,,,
0,1,1,Brasília,*,*,1,0,1,1,0
1,2,1,Pesquisa_Datafolha,(A0*,A0,2,1,4,3,35
2,3,1,publicada,*,A0,2,1,4,3,35
3,4,1,hoje,*),A0,2,1,4,3,35
4,5,1,revela,(V*),V,3,4,5,1,126
5,6,1,um,(A1*,A1,4,5,32,27,181
6,7,1,dado,*,A1,4,5,32,27,181
7,8,1,supreendente,*,A1,4,5,32,27,181
8,9,1,:,*,A1,4,5,32,27,181


## 2. Gets encodings

Propbank Encoder holds an indexed version of propbank dataset an answers to FOUR different dataformats: 
* CAT: this is the raw categorical data.
* EMB: tokens are embedding using GloVe embeddings.
* HOT: onehot encoding of the words and tokens.
* IDX: dense indexed representations.

In [4]:
# LOAD ENCODER
propbank_encoder = PropbankEncoder.recover(PROPBANK_WAN50_PATH)
db = propbank_encoder.db
lex2idx = propbank_encoder.lex2idx
idx2lex = propbank_encoder.idx2lex

# FOR TEXTUAL DATA ONLY
tok2idx = propbank_encoder.tok2idx
lex2tok = propbank_encoder.lex2tok
idx2word = propbank_encoder.idx2word
embeddings = propbank_encoder.embeddings

n_targets = len(lex2idx['T'])

In [5]:
print('attributes\t',
       len(db),
      '\n',             
      'records\t',
       len(db['ARG'].keys()))

attributes	 43 
 records	 138378


In [6]:
def filter_type(ds_type, db):
    '''Filters only records from train dataset
    '''
    lb, ub = get_db_bounds(ds_type)

    sel_keys_ = {key_ for key_, prop_ in db['P'].items() if prop_ >= lb and prop_ < ub}

    return {
                attr_:{ idx_: i_
                        for idx_, i_ in dict_.items() if idx_ in sel_keys_
                      }        
                for attr_, dict_  in db.items()
            }

def make_propositions_dict(db):
    '''Reindex db by propositions creating a nested dict in which the
        outer key is the proposition        
    '''
    
    triple_list = []
    prev_idx = -1
    prev_prop = -1
    for idx, prop in db['P'].items():
        
        if prev_prop != prop:
            if idx > 0:
                ub = prev_idx
                triple_list.append((lb, ub, prev_prop))
            lb = idx
        prev_prop = prop
        prev_idx = idx
    triple_list.append((lb, prev_idx, prev_prop))
            

        
    prop_set = set(db['P'].values())
    return { prop_:
                    {
                        attr_:{ idx_: dict_[idx_]
                                for idx_ in range(lb_, ub_ + 1, 1)
                          }        
                        for attr_, dict_ in db.items() if attr_ not in ('P',)
                    }
             for lb_, ub_, prop_ in  triple_list
            }, {prop_: ub_ - lb_ + 1 for lb_, ub_, prop_ in  triple_list}   


def numpfy_propositions_dict(prop_dict, proplen_dict):
    '''Converts inner dict examples into numpy arrays
    '''
    prop_dict_ = defaultdict(dict)    
    for prop, columns_dict in prop_dict.items():
        len_ = proplen_dict[prop]
        shape_ = (len_, 1)
        for column, values_dict in columns_dict.items():
            tuple_list = [idx_value 
                          for idx_value in values_dict.items()]
            
            tuple_list = sorted(tuple_list, key=lambda x: x[0])            
            # Converts lexicon (raw/indexed) into token (embedded/indexed)
            if (('FORM' in column) or ('LEMMA' in column)):
                values_list = [tok2idx[lex2tok[idx2word[tuple_[1]]]]                
                                   for tuple_ in tuple_list]
            else:
                values_list = [tuple_[1] for tuple_ in tuple_list]
            
            prop_dict_[prop][column]  = np.array(values_list).reshape(shape_)
    
    return prop_dict_        


In [7]:
traindb  = filter_type('train', db)
print('attributes\t',
       len(traindb),
      '\n',             
      'records\t',
       len(traindb['ARG'].keys()),
       '\n',             
      'vocab\t',
        max([form for _, form in traindb['FORM'].items()]))

attributes	 43 
 records	 128258 
 vocab	 14817


In [8]:
prop_dict, proplen_dict = make_propositions_dict(traindb)
print('attributes\t',
       len(prop_dict[1]) + 1,
      '\n',             
      'records\t',
       sum([len(d['ARG']) for p, d in prop_dict.items()]),
        '\n',             
      'vocab\t',
        max([form for _, prop in prop_dict.items() for _, form in prop['FORM'].items()]))

attributes	 43 
 records	 128258 
 vocab	 14817


In [9]:
prop_dict1 = numpfy_propositions_dict(prop_dict, proplen_dict)
print('attributes\t',
       len(prop_dict1[1]) + 1,
      '\n',             
      'records\t',
       sum([len_ for _, len_ in proplen_dict.items()]),
        '\n',             
      'vocab\t',
        max([max(form) for _, prop in prop_dict1.items() for form in prop['FORM']]))

attributes	 43 
 records	 128258 
 vocab	 12439


In [10]:
def get_inputs(db1, propid):
    '''Generate inputs
    '''
    propdb = db1[propid] # nested dict of columns and idx value
    if 'CHUNK_SPACE' not in propdb:
        proplen = len(propdb['ID'])
        propdb['CHUNK_SPACE'] = generate_chunk_space(proplen)

    # Replaces word with tokens
    word    = propdb['FORM']
    ctx_p_left  = propdb['FORM_CTX_P-1']
    ctx_p0  = propdb['FORM_CTX_P+0']
    ctx_p_right  = propdb['FORM_CTX_P+1']
    
    marker  = propdb['MARKER']
    pos     = propdb['GPOS']
    chunk_type  = propdb['T']
    chunk_start, chunk_finish = propdb['CHUNK_SPACE']
    
    return word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish
            
def generate_chunk_space(n):
    '''Generates all possible spaces for chunks
    '''
    start_list = []
    end_list = []
    for i in range(n):
        for j in range(i,n,1):
            start_list.append(i)
            end_list.append(j+1)
    shape_ = (len(start_list), 1)
    start_ = np.array(start_list).reshape(shape_)
    finish_ = np.array(end_list).reshape(shape_)
    return start_, finish_
            

def get_outputs(db1, propid, n_targets):
    ''' Generate outputs
    '''
    propdb = db1[propid] # nested dict of columns and idx value
    if 'OUTPUTS' not in propdb: 
        id_type = np.concatenate(
            ( propdb['CHUNK_CANDIDATE_ID'], propdb['T']), axis=1
        )

        id_type = np.unique( id_type, axis=0)
        propdb['OUTPUTS'] = id_type[:,0] * n_targets + id_type[:,1]

    return propdb['OUTPUTS']

In [11]:
%%timeit
propid = 1120
word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
targets = get_outputs(prop_dict1, propid, n_targets)
# worst proposition 1120 size 92!

816 ns ± 15.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


 ## MODEL

In [12]:
propid = 1
word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1,  propid)
# import code; code.interact(local=dict(globals(), **locals()))
targets = get_outputs(prop_dict1, propid, n_targets)
print(targets)
_start  = np.repeat(chunk_start, n_targets)
_finish = np.repeat(chunk_finish, n_targets)
print(_start)
print(_finish)
print(list(zip(_start[targets].flatten(), _finish[targets].flatten())))


[    0   666  2412  3441 10640]
[ 0  0  0 ... 32 32 32]
[ 1  1  1 ... 33 33 33]
[(0, 1), (1, 4), (4, 5), (5, 32), (32, 33)]


In [13]:
import struct_perc.colored_weighted_interval_scheduling as cwis
import struct_perc.weighted_interval_scheduling as wis
import struct_perc.utils as spu

 ## Tensorflow Graph

In [14]:
# Parameters
vocab_size = len(embeddings)
embed_size = 50

n_pos = len(lex2idx['GPOS'])
n_classes = len(lex2idx['T'])
n_features = embed_size * 4 + 1 + n_pos

n_hidden = 100
# W_shape = (n_features, n_hidden)
W_shape = (n_hidden, n_hidden)
b_shape = (1, n_hidden)

W_interval_shape = (2 * n_hidden, n_classes)
# W_shape = (hidden_features, n_classes)
b_interval_shape = (1, n_classes)

# word index 
X_words = tf.placeholder(tf.int64, shape=(None,1), name='word')

# predicate context index (left, predicate, right)
X_ctx_p_left = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p_left')
X_ctx_p = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p0')
X_ctx_p_right = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p_right')

# POS tagging feature
X_pos = tf.placeholder(tf.int64, shape=(None,1), name='gpos')
X_marker = tf.cast( tf.placeholder(tf.int64, shape=(None,1), name='marker'), tf.float32 )
EMBS = tf.Variable(embeddings, trainable=False)

# Embedded representation
with tf.variable_scope("features"):
    EMBS_words = tf.gather_nd(EMBS, X_words, name='word_features')

    EMBS_ctx_pleft = tf.gather_nd(EMBS, X_ctx_p_left, name='EMBS_ctx_pleft')
    EMBS_ctx_p0 = tf.gather_nd(EMBS, X_ctx_p, name='EMBS_ctx_p0')
    EMBS_ctx_pright = tf.gather_nd(EMBS, X_ctx_p_right, name='EMBS_ctx_pright')

    X_pos_flat = tf.reshape(X_pos, [-1], name='gpos_flat')
    X_pos_onehot = tf.one_hot(X_pos_flat, depth=n_pos, name='gpos_onehot')

    X = tf.concat((EMBS_words, EMBS_ctx_pleft, EMBS_ctx_p0,
                   EMBS_ctx_pright, X_pos_onehot, X_marker),
                  axis=1, name='X')
    X_batch = tf.expand_dims(X, 0)

with tf.variable_scope('gru', reuse=tf.AUTO_REUSE):

    fw = tf.nn.rnn_cell.GRUCell(num_units=n_hidden / 2)
    bw = tf.nn.rnn_cell.GRUCell(num_units=n_hidden / 2)
    
    Wo = tf.Variable(tf.truncated_normal(W_shape, stddev=1.0 / np.sqrt(n_features * n_hidden)), name='W' )
    bo = tf.Variable(tf.zeros(b_shape, dtype=tf.float32), name='b')

    hidden_outputs, states = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=fw,
        cell_bw=bw,
        inputs=X_batch,
        dtype=tf.float32
    )
    hidden_fw, hidden_bw = hidden_outputs
    Ho = tf.concat((tf.squeeze(hidden_fw, axis=0) ,tf.squeeze(hidden_fw, axis=0)),axis=1)

    Z = tf.nn.tanh( tf.matmul( Ho, Wo ) + bo, name='hidden_layer' )

# Those are the interval parameters
with tf.variable_scope("interval"):
    W_interval = tf.Variable(tf.random_normal(W_interval_shape, mean=0.0, stddev=1.0 / np.sqrt(1.0 * n_hidden * n_classes)), name='W_interval')
    b_interval = tf.Variable(tf.zeros(b_interval_shape, dtype=tf.float32), name='b_interval')
    
    # begin of interval
    IntervalStart = tf.placeholder(tf.int32, shape=(None,1))
    # end of interval
    IntervalFinish = tf.placeholder(tf.int32, shape=(None,1))

# features from intervals
IntervalFinishZ = tf.gather_nd(Z, IntervalFinish-1)
IntervalStartZ = tf.gather_nd(Z, IntervalStart)

IntervalZ = tf.concat((IntervalFinishZ, IntervalStartZ), axis=1)
IntervalScores = tf.matmul(IntervalZ, W_interval) + b_interval

ScoresFlat = tf.reshape(IntervalScores, (-1,1)) # column array n_classes * ((len + 1 ) * len) / 2 
ScoresMean = tf.reduce_mean(ScoresFlat) # scalar
ScoresDiff = ScoresFlat - ScoresMean   # centralize data --> mean zero

ScoresStd = tf.sqrt(tf.reduce_sum(ScoresDiff * ScoresDiff))
ScoresOp = ScoresDiff /( ScoresStd + 1e-8 )

 ## Tensorflow test session

In [15]:
propid  = 1
words, ctx_p_left, ctx_p0, ctx_p_rigxht, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
targets = get_outputs(prop_dict1, propid, n_targets)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    arg_list = [Z, IntervalZ, IntervalStartZ, IntervalFinishZ, W_interval, b_interval, IntervalScores]
    data_list = sess.run(arg_list, feed_dict={
        X_words:words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,                
        X_pos:gpos,
        IntervalStart: chunk_start,
        IntervalFinish: chunk_finish
    })
    ZZ, scores, start_scores, finish_scores, Wi, bi, inteval_scores = data_list
    # flat gives the score for each candidate

print(ZZ.shape, scores.shape, start_scores.shape, finish_scores.shape, Wi.shape, bi.shape, inteval_scores.shape, targets)


(33, 100) (561, 200) (561, 100) (561, 100) (200, 19) (1, 19) (561, 19) [    0   666  2412  3441 10640]


## Testing prediction

In [16]:
def pred(sess, x_words, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos, x_chunk_start, x_chunk_finish):
    scores = sess.run(ScoresOp,feed_dict={
        X_words: x_words,
        X_ctx_p_left: x_ctx_p_left,
        X_ctx_p: x_ctx_p0,
        X_ctx_p_right: x_ctx_p_right,        
        X_marker: x_marker,                
        X_pos:x_pos,
        IntervalStart: x_chunk_start,
        IntervalFinish: x_chunk_finish
    })
    # scores is a ((proplen + 1) * (proplen) / 2) * n_classes  
    starts = np.repeat(x_chunk_start, n_classes).reshape((-1,1))
    ends = np.repeat(x_chunk_finish, n_classes).reshape((-1,1))
    
    ck_len = len(x_chunk_start)
    colors = np.array(list(np.arange(n_classes))*ck_len)

    # Finds best allocation given the scores and the chunk_space
    r_int = cwis.compute_schedule(starts.flatten(), ends.flatten(), scores, colors) # index of the cadidates of predicted solution
    r_ext = list(zip(starts[r_int].flatten(),ends[r_int].flatten(), colors[r_int].flatten())) # from integer to triple
    return r_int, r_ext

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
p, pe = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)
print(p)
print(pe)

[10641 10603 10546 10470 10375 10261 10128  9976  9805  9615  9406  9178
  8931  8665  8380  8076  7753  7411  7050  6670  6271  5853  5416  4960
  4485  3991  3478  2946  2395  1825  1236   628    18]
[(32, 33, 1), (31, 32, 1), (30, 31, 1), (29, 30, 1), (28, 29, 1), (27, 28, 1), (26, 27, 1), (25, 26, 1), (24, 25, 1), (23, 24, 1), (22, 23, 1), (21, 22, 1), (20, 21, 1), (19, 20, 1), (18, 19, 1), (17, 18, 1), (16, 17, 1), (15, 16, 1), (14, 15, 1), (13, 14, 1), (12, 13, 1), (11, 12, 1), (10, 11, 1), (9, 10, 1), (8, 9, 1), (7, 8, 1), (6, 7, 1), (5, 6, 1), (4, 5, 1), (3, 4, 1), (2, 3, 1), (1, 2, 1), (0, 1, 18)]


## Training code

In [18]:
# indices of the correct intervals
T = tf.placeholder(tf.int32, shape=(None,), name='T')
L = tf.placeholder(tf.int32, shape=(), name='L')

T_flat = tf.reshape(T, (-1,1), name='T_flat') # column array
MarginFactor = tf.placeholder(tf.float32)
MarginIndex = tf.ones(tf.shape(T_flat)) * MarginFactor
MarginScores = tf.scatter_nd(T_flat, -MarginIndex, tf.shape(ScoresOp)) # oppposite of gather_nd
ScoresWithMargin = ScoresOp + MarginScores


In [19]:
# indices containing the predicted labels from Weighted Interval Scheduling
Y = tf.placeholder(tf.int32, shape=(None,), name='predictions')
Y_flat = tf.reshape(Y, (-1, 1))

# score da estrutura predita
ScoreY = tf.gather_nd(ScoresWithMargin, Y_flat, name='predicted_score')
# score da estrutura correta
ScoreT = tf.gather_nd(ScoresWithMargin, T_flat, name='target_score')

# função de custo do perceptron estruturado
CostOp = tf.reduce_sum(ScoreY) - tf.reduce_sum(ScoreT)

# gradiente descendente no custo do perceptron estruturado
Optimizer = tf.train.AdamOptimizer(0.001)
TrainOp = Optimizer.minimize(CostOp)

/Users/Varela/.venv/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [20]:
predictions, _ = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)
print(targets.shape)
print(targets)
print(predictions.shape)
print(predictions)

(5,)
[    0   666  2412  3441 10640]
(33,)
[10641 10603 10546 10470 10375 10261 10128  9976  9805  9615  9406  9178
  8931  8665  8380  8076  7753  7411  7050  6670  6271  5853  5416  4960
  4485  3991  3478  2946  2395  1825  1236   628    18]


 ## Testing cost operation

In [21]:
predictions, chunk_ext = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)

interval_start = np.repeat(chunk_start, n_classes).reshape((-1,1))
interval_finish = np.repeat(chunk_finish, n_classes).reshape((-1,1))

scores_, cost_ = sess.run([ScoresOp, CostOp], feed_dict={
    X_words: words,
    X_ctx_p_left: ctx_p_left,
    X_ctx_p: ctx_p0,
    X_ctx_p_right: ctx_p_right,        
    X_marker: marker,        
    X_pos: gpos,
    IntervalStart: interval_start,
    IntervalFinish: interval_finish,
    T: targets.flatten(),
    Y: predictions.flatten(),        
    L: proplen_dict[propid],
    MarginFactor:0.01})


print(scores_.shape)
print(cost_)
print(np.max(predictions))
print(np.sum(scores_[predictions]) - np.sum(scores_[targets]))

(202521, 1)
0.12661356
10641
0.07661356


In [22]:
colors = np.array(list(np.arange(n_classes))*len(chunk_start))
r_int = cwis.compute_schedule(interval_start.flatten(), interval_finish.flatten(), scores_, colors) 
print(np.mean(scores_[r_int]), ' ', np.mean(scores_[targets]))

0.0031814256   6.026579e-05


In [23]:
np.sum(scores_[predictions].flatten())

0.07691489

In [24]:
np.sum(scores_[targets].flatten())

0.00030132895

In [25]:
print(sorted(predictions))
print(sorted(r_int))

[18, 628, 1236, 1825, 2395, 2946, 3478, 3991, 4485, 4960, 5416, 5853, 6271, 6670, 7050, 7411, 7753, 8076, 8380, 8665, 8931, 9178, 9406, 9615, 9805, 9976, 10128, 10261, 10375, 10470, 10546, 10603, 10641]
[18, 645, 1253, 1827, 2397, 2948, 3480, 3993, 4487, 4962, 5418, 5855, 6273, 6672, 7052, 7413, 7755, 8078, 8382, 8667, 8933, 9180, 9408, 9617, 9807, 9978, 10130, 10263, 10377, 10472, 10548, 10605, 10643]


In [26]:
propid  = 1
words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
targets = get_outputs(prop_dict1, propid, n_targets)

starts = np.repeat(chunk_start,n_classes).reshape((-1,1))
ends = np.repeat(chunk_finish, n_classes).reshape((-1,1))

scores_ = sess.run(ScoresOp, feed_dict={
    X_words: words,
    X_ctx_p_left: ctx_p_left,
    X_ctx_p: ctx_p0,
    X_ctx_p_right: ctx_p_right,        
    X_marker: marker,        
    X_pos: gpos,
    IntervalStart: chunk_start,
    IntervalFinish: chunk_finish,
    T: targets.flatten(),
    Y: predictions.flatten(),        
    MarginFactor:0.01
})

colors = np.repeat(np.arange(n_classes), len(chunk_start))
predictions = cwis.compute_schedule(starts.flatten(), ends.flatten(), scores_, colors) 
t_flat, margin_index, shape_scores = sess.run([T_flat, -MarginIndex, tf.shape(ScoresOp)], feed_dict={
    X_words: words,
    X_ctx_p_left: ctx_p_left,
    X_ctx_p: ctx_p0,
    X_ctx_p_right: ctx_p_right,        
    X_marker: marker,        
    X_pos: gpos,
    IntervalStart: chunk_start,
    IntervalFinish: chunk_finish,
    T:targets.flatten(),
    Y:predictions.flatten(),
    MarginFactor:0.01
})


cost_ = sess.run(CostOp, feed_dict={
    X_words: words,
    X_ctx_p_left: ctx_p_left,
    X_ctx_p: ctx_p0,
    X_ctx_p_right: ctx_p_right,        
    X_marker: marker,        
    X_pos: gpos,
    IntervalStart: chunk_start,
    IntervalFinish: chunk_finish,
    T:targets.flatten(),
    Y:predictions.flatten(),
    MarginFactor:0.01
})
# print(targets)
# print(scores_.shape)
# print(t_flat, t_flat.shape)
# print(margin_index, margin_index.shape)
# print(shape_scores, shape_scores.shape)
print(np.sum(scores_[predictions]) - np.sum(scores_[targets]))

0.63616747


In [27]:
print(scores_.shape, predictions.shape, words.shape, targets.shape)

(10659, 1) (33,) (33, 1) (5,)


In [28]:
print(targets)
print(predictions)

[    0   666  2412  3441 10640]
[10641 10603 10546 10470 10375 10261 10128  9976  9805  9615  9406  9178
  8931  8665  8380  8076  7753  7411  7050  6670  6271  5853  5416  4960
  4485  3991  3478  2946  2395  1825  1236   628    18]


## Training Single Proposition

In [29]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

propid = 1
words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
targets = get_outputs(prop_dict1, propid, n_targets)

starts = np.repeat(chunk_start,n_classes).reshape((-1,1))
ends = np.repeat(chunk_finish, n_classes).reshape((-1,1))

for i in range(10000):
    
    scores_ = sess.run(ScoresOp, feed_dict={
        X_words: words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,        
        X_pos: gpos,
        IntervalStart: chunk_start,
        IntervalFinish: chunk_finish,
        T: targets.flatten(),
        Y: predictions.flatten(),        
        MarginFactor:0.01
    })
    
    predictions, chunk_ext = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)    
    cost_ = sess.run(CostOp, feed_dict={
        X_words: words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,        
        X_pos: gpos,
        IntervalStart: chunk_start,
        IntervalFinish: chunk_finish,
        T: targets.flatten(),
        Y: predictions.flatten(),        
        MarginFactor:0.01
    })
    
    predictions_score = np.sum(scores_[predictions])
    targets_score = np.sum(scores_[targets])
        
    sess.run(TrainOp, feed_dict={
        X_words: words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,        
        X_pos: gpos,
        IntervalStart: chunk_start,
        IntervalFinish: chunk_finish,
        T: targets.flatten(),
        Y: predictions.flatten(),        
        MarginFactor:0.01})
    
    colors = np.repeat(np.arange(n_classes), len(chunk_start))
    predictions = cwis.compute_schedule(starts.flatten(), ends.flatten(), scores_, colors) 

    targets_set = set(targets.flatten())
    predictions_set = set(predictions.flatten())
    yp_common = targets_set.intersection(predictions_set)
    yp_total = targets_set.union(predictions_set)
    
    acc_int = len(yp_common)/len(yp_total)
    
    if i % 25 == 0:
        print(acc_int, ' ', cost_)
    
    if cost_ < 0:
        break

    if acc_int == 1:
        print(acc_int, ' ', cost_, ' learnt at epoch ', i)
        break


0.0   0.5601393
0.0   0.29042652
0.02702702702702703   0.20950466
0.05555555555555555   0.17154808
0.02702702702702703   0.1409531
0.08823529411764706   0.061511092
0.09375   0.035328064
0.42857142857142855   0.020294216


In [30]:
targets_list = list(zip(starts[targets].flatten(), ends[targets].flatten(), colors[targets]))

print(sorted(targets_list, key=lambda x: x[0]))

[(0, 1, 0), (1, 4, 1), (4, 5, 4), (5, 32, 6), (32, 33, 18)]


In [31]:
predictions_list = list(zip(starts[predictions].flatten(), ends[predictions].flatten(), colors[predictions]))

print(sorted(predictions_list, key=lambda x: x[0]))

[(0, 1, 0), (1, 4, 1), (4, 5, 4), (5, 32, 6), (32, 33, 18)]


In [32]:
# this should be equal to previous cell result predicitin doesn't changed the weights
predictions1, chunk_ext = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)    
predictions_test_list = list(zip(starts[predictions1].flatten(), ends[predictions1].flatten(), colors[predictions1]))

print(sorted(predictions_test_list, key= lambda x: x[0]))

[(0, 1, 0), (1, 4, 1), (4, 5, 4), (5, 32, 6), (32, 33, 18)]


 ## ConLL evaluation scripts

In [35]:
def tag_to_conll(sess, prop_dict, propid, idx2lex):
    gold_list = []
    eval_list = []
        
    words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict, propid)
    targets = get_outputs(prop_dict, propid, n_targets)

    predictions, chunk_ext = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)    


    n_words = len(words)
    default_ck_list_ = [(i,i + 1, '*') for i in range(n_words)]
    
    pred_array = prop_dict[propid]['PRED']
    pred_array = pred_array.flatten()
    
    arg_array = prop_dict[propid]['ARG']
    arg_array = arg_array.flatten()
    
    pred_list = [idx2lex['PRED'][i] for i in pred_array.tolist()]
    gold_list_ = [idx2lex['ARG'][i] for i in arg_array.tolist()]
     
    gold_list += list(zip(pred_list, gold_list_))

    
    ck_list_ = []     
    for triple_ in sorted(chunk_ext, key= lambda x: x[0]):
        lb, ub, ckid = triple_
        # filters default value
        default_ck_list_ = [
            dck_
            for dck_ in default_ck_list_ if dck_[0] < lb or dck_[1] > ub
        ]
        ck_list_.append((lb, ub, idx2lex['T'][ckid]))        

    ck_list_ = default_ck_list_ + ck_list_ 

    arg_list_ = []
    for triple_ in sorted(ck_list_, key= lambda x: x[0]):
        lb, ub, cktype = triple_
        flat_list_ = [ cktype if i == lb else '*' for i in range(lb, ub) ]
            
        if cktype != '*':
            flat_list_[0] = '({:}*'.format(flat_list_[0])
            flat_list_[-1] = '{:})'.format(flat_list_[-1])
        arg_list_ += flat_list_
        
    eval_list += list(zip(pred_list, arg_list_))
    eval_list.append(None)
    gold_list.append(None)
    # Change to use CoNLL2004
    gold_list = propbankbr_arg2se(gold_list)
    eval_list = propbankbr_arg2se(eval_list)
    return gold_list, eval_list

testamos o modelo na frase usada para treino

In [36]:
propid = 1
gold_list, eval_list = tag_to_conll(sess, prop_dict1, propid, idx2lex)
for i in range(proplen_dict[propid]):
    if gold_list[i] and eval_list[i]:
        print('{:}\t{:}\t{:}\t{:}'.format(*gold_list[i], *eval_list[i]))
    else:
        print('\n')
    

print(lex2idx['T'])

-	*	-	*
-	(A0*	-	(A0*
-	*	-	*
-	*A0)	-	*A0)
revelar	(V*V)	revelar	(V*V)
-	(A1*	-	(A1*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*	-	*
-	*A1)	-	*A1)
-	*	-	*
OrderedDict([('*', 0), ('A0', 1), ('A1', 2), ('A2', 3), ('A3', 4), ('A4', 5), ('AM-ADV', 6), ('AM-CAU', 7), ('AM-DIR', 8), ('AM-DIS', 9), ('AM-EXT', 10), ('AM-LOC', 11), ('AM-MNR', 12), ('AM-NEG', 13), ('AM-PNC', 14), ('AM-PRD', 15), ('AM-REC', 16), ('AM-TMP', 17), ('V', 18)])


In [37]:
def evaluate(gold_list, eval_list, verbose=True):
    gold_path = 'train_gold.props'    
    eval_path = 'train_eval.props'

    with open(gold_path, mode='w') as f:        
        for tuple_ in gold_list:
            if tuple_ is None:
                f.write('\n')
            else:
                f.write('{:}\t{:}\n'.format(*tuple_))

    with open(eval_path, mode='w') as f:        
        for tuple_ in eval_list:
            if tuple_ is None:
                f.write('\n')
            else:
                f.write('{:}\t{:}\n'.format(*tuple_))

    pipe = Popen(['perl',PEARL_SRLEVAL_PATH, gold_path, eval_path], stdout=PIPE, stderr=PIPE)

    txt, err = pipe.communicate()
    txt = txt.decode('UTF-8')
    err = err.decode('UTF-8')

    if verbose:
        print(txt)
        with open('train.conll', mode='w') as f:
            f.write(txt)

    # overall is a summary from the list
    # is the seventh line
    lines_list = txt.split('\n')        
    
    # get the numbers from the row 
    overall_list = re.findall(r'[-+]?[0-9]*\.?[0-9]+.', lines_list[6])
    f1 = float(overall_list[-1])

    return f1



In [39]:
propid = 1
gold_list, eval_list = tag_to_conll(sess, prop_dict1, propid, idx2lex)
f1_score = evaluate(gold_list, eval_list, verbose=True)
print('f1_score: ', f1_score)

this is the output Number of Sentences    :           1
Number of Propositions :           1
Percentage of perfect props : 100.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        2       0       0   100.00  100.00  100.00
----------
        A0        1       0       0   100.00  100.00  100.00
        A1        1       0       0   100.00  100.00  100.00
------------------------------------------------------------
         V        1       0       0   100.00  100.00  100.00
------------------------------------------------------------

this is the error 
Number of Sentences    :           1
Number of Propositions :           1
Percentage of perfect props : 100.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        2       0       0   100.00  100.00  100.00
----------
        A0        1       0       0   100.00

In [40]:
import time
start = time.time()
print(1111)
gold_list, eval_list = tag_to_conll(sess, prop_dict1, propid, idx2lex)
f1_score = evaluate(gold_list, eval_list, verbose=True)
end = time.time()
print('evaluate: ', (end-start), 's')
print('f1_score: ', f1_score)

1111
this is the output Number of Sentences    :           1
Number of Propositions :           1
Percentage of perfect props : 100.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        2       0       0   100.00  100.00  100.00
----------
        A0        1       0       0   100.00  100.00  100.00
        A1        1       0       0   100.00  100.00  100.00
------------------------------------------------------------
         V        1       0       0   100.00  100.00  100.00
------------------------------------------------------------

this is the error 
Number of Sentences    :           1
Number of Propositions :           1
Percentage of perfect props : 100.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        2       0       0   100.00  100.00  100.00
----------
        A0        1       0       0   1

## Training

In [41]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

EPOCHS = 1000
NPROPS = 40
# indices = np.arange(config.DATASET_TRAIN_SIZE)
indices = np.arange(NPROPS) + 1

best_score = 0
saver = tf.train.Saver()

iter_dict = {'iteration': 0, 'epochs': 0, 'accuracy': 0.0, 'cost': 999.0, 'f1_score': 0.0, 'best_score': 0.0}
iter_message = 'iteration:{iteration:05d}\tepochs:{epochs:05d}\t' + \
                'accuracy:{accuracy:0.2f}\tcost:{cost:0.2f}\t' + \
                'f1_score:{f1_score:0.2f}\tbest_score:{best_score:0.2f}'

for j in range(EPOCHS):
    np.random.shuffle(indices)
    total_err = 0
    total_size = 0
    total_cost = 0
    gold_list = []
    eval_list = []         
    for i, propid in enumerate(indices):     
        try: 
            words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
            targets = get_outputs(prop_dict1, propid, n_targets)
        except KeyError:
            print(propid)

        predictions, chunk_ext = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_start, chunk_finish)

        _, cost = sess.run([TrainOp, CostOp], feed_dict={
            X_words: words,
            X_ctx_p_left: ctx_p_left,
            X_ctx_p: ctx_p0,
            X_ctx_p_right: ctx_p_right,        
            X_marker: marker,        
            X_pos: gpos,
            IntervalStart: chunk_start,
            IntervalFinish: chunk_finish,
            T: targets.flatten(),
            Y: predictions.flatten(),        
            MarginFactor:0.01})

        predictions_set = set(predictions.tolist())
        targets_set = set(targets.tolist())
        
        union_set = predictions_set.union(targets_set)
        inter_set = predictions_set.intersection(targets_set)
        
        total_err += len(union_set - inter_set)
        total_size += len(union_set)
        total_cost += cost
        
        gold_list_, eval_list_ = tag_to_conll(sess, prop_dict1, propid, idx2lex)
        gold_list += gold_list_
        eval_list += eval_list_

        iter_dict['iteration'] = i + j * len(indices)
        iter_dict['epochs'] = j
        iter_dict['accuracy'] = 1 - total_err/total_size
        iter_dict['cost'] = total_cost

        
    iter_dict['f1_score'] = evaluate(gold_list, eval_list, verbose=False)    
    if iter_dict['f1_score'] > iter_dict['best_score']:
        iter_dict['best_score'] = iter_dict['f1_score']
        evaluate(gold_list, eval_list, verbose=True)    
        save_path = saver.save(sess, "/tmp/model_spn-pt.ckpt")
    
    print(iter_message.format(**iter_dict))        
    
    if iter_dict['best_score'] > 98:
        print('best_score:',iter_dict['best_score'],'exiting')
        break    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :   0.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        1     561      97     0.18    1.02    0.30
----------
        A0        0       0      22     0.00    0.00    0.00
        A1        1     118      38     0.84    2.56    1.27
        A2        0      71       8     0.00    0.00    0.00
        A3        0      86       1     0.00    0.00    0.00
        A4        0      16       0     0.00    0.00    0.00
    AM-ADV        0      10       2     0.00    0.00    0.00
    AM-CAU        0       0       2     0.00    0.00    0.00
    AM-DIS        0      42       3     0.00    0.00    0.00
    AM-LOC        0       0       6     0.00    0.00    0.00
    AM-MNR        0      44       1     0.00    0.00    0.00
    AM-NEG        0      68       4     0.00    0.00    0.00
    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :   0.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        4     532      94     0.75    4.08    1.26
----------
        A0        0      59      22     0.00    0.00    0.00
        A1        1     104      38     0.95    2.56    1.39
        A2        0      29       8     0.00    0.00    0.00
        A3        0      48       1     0.00    0.00    0.00
        A4        0       3       0     0.00    0.00    0.00
    AM-ADV        1      20       1     4.76   50.00    8.70
    AM-CAU        0       0       2     0.00    0.00    0.00
    AM-DIS        0      72       3     0.00    0.00    0.00
    AM-LOC        0      25       6     0.00    0.00    0.00
    AM-MNR        0      39       1     0.00    0.00    0.00
    AM-NEG        1      53       3     1.85   25.00    3.45
    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :   0.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        1     420      97     0.24    1.02    0.39
----------
        A0        0      21      22     0.00    0.00    0.00
        A1        1      81      38     1.22    2.56    1.65
        A2        0      39       8     0.00    0.00    0.00
        A3        0       0       1     0.00    0.00    0.00
        A4        0      26       0     0.00    0.00    0.00
    AM-ADV        0      45       2     0.00    0.00    0.00
    AM-CAU        0       5       2     0.00    0.00    0.00
    AM-DIR        0      12       0     0.00    0.00    0.00
    AM-DIS        0      29       3     0.00    0.00    0.00
    AM-LOC        0      13       6     0.00    0.00    0.00
    AM-MNR        0      14       1     0.00    0.00    0.00
    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :   0.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        4     152      94     2.56    4.08    3.15
----------
        A0        1      38      21     2.56    4.55    3.28
        A1        2      40      37     4.76    5.13    4.94
        A2        0      10       8     0.00    0.00    0.00
        A3        0       2       1     0.00    0.00    0.00
    AM-ADV        0       4       2     0.00    0.00    0.00
    AM-CAU        0       0       2     0.00    0.00    0.00
    AM-DIS        0       6       3     0.00    0.00    0.00
    AM-LOC        1      10       5     9.09   16.67   11.76
    AM-MNR        0       4       1     0.00    0.00    0.00
    AM-NEG        0      18       4     0.00    0.00    0.00
    AM-PNC        0       2       0     0.00    0.00    0.00
    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :   0.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall        9     103      89     8.04    9.18    8.57
----------
        A0        1      20      21     4.76    4.55    4.65
        A1        7      40      32    14.89   17.95   16.28
        A2        0      11       8     0.00    0.00    0.00
        A3        0       0       1     0.00    0.00    0.00
        A4        0       2       0     0.00    0.00    0.00
    AM-ADV        0       2       2     0.00    0.00    0.00
    AM-CAU        0       0       2     0.00    0.00    0.00
    AM-DIS        0       0       3     0.00    0.00    0.00
    AM-LOC        1      16       5     5.88   16.67    8.70
    AM-MNR        0       0       1     0.00    0.00    0.00
    AM-NEG        0       2       4     0.00    0.00    0.00
    

...

iteration:04759	epochs:00118	accuracy:0.76	cost:0.30	f1_score:86.46	best_score:86.46
this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :  75.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall       87       8      11    91.58   88.78   90.16
----------
        A0       22       0       0   100.00  100.00  100.00
        A1       37       2       2    94.87   94.87   94.87
        A2        8       1       0    88.89  100.00   94.12
        A3        1       0       0   100.00  100.00  100.00
    AM-ADV        1       0       1   100.00   50.00   66.67
    AM-CAU        2       0       0   100.00  100.00  100.00
    AM-DIR        0       1       0     0.00    0.00    0.00
    AM-DIS        3       0       0   100.00  100.00  100.00
    AM-LOC        6       1       0    85.71  100.00   92.31
    AM-MNR        0       0       1     0

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :  70.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall       86       9      12    90.53   87.76   89.12
----------
        A0       21       3       1    87.50   95.45   91.30
        A1       37       1       2    97.37   94.87   96.10
        A2        8       2       0    80.00  100.00   88.89
        A3        1       0       0   100.00  100.00  100.00
    AM-ADV        2       2       0    50.00  100.00   66.67
    AM-CAU        2       0       0   100.00  100.00  100.00
    AM-DIS        3       0       0   100.00  100.00  100.00
    AM-LOC        6       1       0    85.71  100.00   92.31
    AM-MNR        1       0       0   100.00  100.00  100.00
    AM-NEG        0       0       4     0.00    0.00    0.00
    AM-PRD        1       0       0   100.00  100.00  100.00
    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :  85.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall       92       8       6    92.00   93.88   92.93
----------
        A0       20       4       2    83.33   90.91   86.96
        A1       36       3       3    92.31   92.31   92.31
        A2        8       0       0   100.00  100.00  100.00
        A3        1       0       0   100.00  100.00  100.00
    AM-ADV        2       0       0   100.00  100.00  100.00
    AM-CAU        2       0       0   100.00  100.00  100.00
    AM-DIS        3       0       0   100.00  100.00  100.00
    AM-LOC        5       0       1   100.00   83.33   90.91
    AM-MNR        1       0       0   100.00  100.00  100.00
    AM-NEG        4       1       0    80.00  100.00   88.89
    AM-PRD        1       0       0   100.00  100.00  100.00
    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :  65.00

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall       80      19      18    80.81   81.63   81.22
----------
        A0       18       6       4    75.00   81.82   78.26
        A1       31       5       8    86.11   79.49   82.67
        A2        7       4       1    63.64   87.50   73.68
        A3        1       0       0   100.00  100.00  100.00
    AM-ADV        1       0       1   100.00   50.00   66.67
    AM-CAU        2       0       0   100.00  100.00  100.00
    AM-DIS        3       1       0    75.00  100.00   85.71
    AM-LOC        5       0       1   100.00   83.33   90.91
    AM-MNR        0       2       1     0.00    0.00    0.00
    AM-NEG        3       0       1   100.00   75.00   85.71
    AM-PRD        1       0       0   100.00  100.00  100.00
    

this is the output Number of Sentences    :          40
Number of Propositions :          40
Percentage of perfect props :  92.50

              corr.  excess  missed    prec.    rec.      F1
------------------------------------------------------------
   Overall       94       4       4    95.92   95.92   95.92
----------
        A0       19       3       3    86.36   86.36   86.36
        A1       38       1       1    97.44   97.44   97.44
        A2        8       0       0   100.00  100.00  100.00
        A3        1       0       0   100.00  100.00  100.00
    AM-ADV        2       0       0   100.00  100.00  100.00
    AM-CAU        2       0       0   100.00  100.00  100.00
    AM-DIS        3       0       0   100.00  100.00  100.00
    AM-LOC        6       0       0   100.00  100.00  100.00
    AM-MNR        1       0       0   100.00  100.00  100.00
    AM-NEG        4       0       0   100.00  100.00  100.00
    AM-PRD        1       0       0   100.00  100.00  100.00
    